Poisson Equation on a Unit Square with Dirichlet Boundary Conditions Solver

Polar Poisson equation: $u_{rr}+\frac{1}{r}u_r+u_{zz} = -f$ 

In [31]:
import basix.ufl
import ufl

from mpi4py import MPI

import matplotlib.pyplot as plt
import numpy as np
import pyvista


from dolfinx import fem, mesh, plot, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
import dolfinx

mesh = mesh.create_unit_square(MPI.COMM_WORLD, 30, 30)
V = fem.functionspace(mesh, ("Lagrange", 1))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)


In [32]:
u_exact = lambda x: np.exp(-x[0]**2)
u_ex = fem.Function(V)
u_ex.interpolate(u_exact)
#u_ex = fem.Constant(mesh, default_scalar_type(1))
def boundary_inner(x):
    return np.isclose(x[0], 0)

def boundary_outer(x):
    return np.isclose(x[0], 1)

tdim = mesh.topology.dim
fdim = tdim - 1
mesh.topology.create_connectivity(fdim, tdim)
facets_inner = dolfinx.mesh.locate_entities_boundary(mesh, fdim, boundary_inner)
facets_outer = dolfinx.mesh.locate_entities_boundary(mesh, fdim, boundary_outer)
dofs_inner = fem.locate_dofs_topological(V, fdim, facets_inner)
dofs_outer = fem.locate_dofs_topological(V, fdim, facets_outer)

bc_inner = fem.dirichletbc(u_ex, dofs_inner, )
bc_outer = fem.dirichletbc(u_ex, dofs_outer,)
bcs =[bc_inner,bc_outer]

In [33]:
source = lambda x: -np.exp(-x[0]**2)*(-2+4*x[0]**2)+2*np.exp(-x[0]**2)
#nuem = lambda x: 2*np.exp(1)*(1-x[1])*x[1]
#g = fem.Function(V)
#g.interpolate(nuem)
g = fem.Constant(mesh, default_scalar_type(0))
f = fem.Function(V)
#f = fem.Constant(mesh, default_scalar_type(2))
f.interpolate(source)
x = ufl.SpatialCoordinate(mesh)
a = ufl.dot(ufl.grad(u), ufl.grad(v))*x[0]* ufl.dx
L = f * v *x[0]* ufl.dx - g * v * ufl.ds 

problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [34]:

if u_exact == None:
    pass
else: 
    if callable(u_exact):
        V2 = fem.functionspace(mesh, ("Lagrange", 2))
        uex = fem.Function(V2)
        uex.interpolate(u_exact)
        uex_1 = fem.Function(V)
        uex_1.interpolate(uex)
        error_max = np.max(np.abs(uex_1.x.array-uh.x.array)) 
        error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)
    else:
        uex = fem.Constant(mesh, default_scalar_type(u_exact))
        uex_1 = u_exact
        error_max = np.max(np.abs([uex_1]*len(uh.x.array)-uh.x.array)) 
        error_max = MPI.COMM_WORLD.allreduce(error_max, op=MPI.MAX)   
    L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
    error_local = fem.assemble_scalar(L2_error)
    error_L2 = np.sqrt(mesh.comm.allreduce(error_local, op=MPI.SUM))

    # Only print the error on one process
    if mesh.comm.rank == 0:
        print(f"Error_L2 : {error_L2:.10e}")
        print(f"Error_max : {error_max:.10e}")

Error_L2 : 2.3190316059e-04
Error_max : 3.9090896899e-04


In [35]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()

Widget(value='<iframe src="http://localhost:35235/index.html?ui=P_0x7f8b5f077b10_10&reconnect=auto" class="pyv…

In [36]:
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()

Widget(value='<iframe src="http://localhost:35235/index.html?ui=P_0x7f8b5f0779d0_11&reconnect=auto" class="pyv…